In [ ]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import csv
import time


job_list=[]
link_list=[]
skill_by_category={}
skill_by_category_list=[]

skill_by_category_all={}
skill_by_category_all_list=[]

#by_page_list_all=[]
by_page_list={}
by_page_list_all=[]
all_page_list=[]



#for zz in range (2, 3):

    
kk=0
for page_number in range (kk, kk+3):
    
    del skill_by_category
    skill_by_category={}

    main_url="http://www.itjobswatch.co.uk"
    my_url = ("https://www.itjobswatch.co.uk/default.aspx?page={}".format(page_number) + "&sortby=0&orderby=0&q=&id=0&lid=2618")

    page = requests.get(my_url).text

    soup=BeautifulSoup (page, "html.parser")
    
    #JOB LIST
    for job in soup.findAll("table", attrs={"class":"results"}):
        for kk in job.findAll("td", "c2"):
            job_list.append(kk.text)
    #LINK LIST 
    for aa in soup.findAll("td", "c2"):
        for bb in aa.findAll("a"):
            link_list.append(main_url + bb.get("href"))        
    #TOP 30 LIST (IN EVERY SUB-PAGE)
    for linknumber in range(0, len(link_list)):
        page1 = requests.get(link_list[linknumber]).text
        soup1=BeautifulSoup (page1, "html.parser")
        ####################### TOP 30 
        tt=[]
        tv=[]
        top_30=[]
        
        allinfo= soup1.findAll("table", attrs={"class":"tab"})
        for aa in allinfo[1].findAll("td", attrs={"class":"tabRelatedGroupsContainer"}):
            for bb in aa.findAll("td", "cd"):
                for cc in bb.findAll("a"):
                    tt.append(cc.text) # Top 30 texts
            
            for dd in aa.findAll("td", "cp"):
                tv.append(dd.text) # Top 30 values
                
        for i in range(0, len(tt)):
            top_30.append([tt[i], tv[i]]) # Top 30 list
       ############################# SKILL SETS
        skill_by_category_all_list=[]

        for aa in allinfo[2].findAll("td", attrs={"class":"tabRelatedGroupsContainer"}):
            skill_set_text=[]
            skill_set_value=[]
            skill_by_category_list=[]
            skill_set_title=[]
            
            for ab in aa.findAll("th", attrs={"colspan":"3"}):
                skill_set_title.append(ab.text)
            for bb in aa.findAll ("td", "cd"):
                skill_set_text.append(bb.a.text)
            for  cc in aa.findAll ("td", "cp"):
                skill_set_value.append(cc.text)
            for ii in range (0, len(skill_set_text)):
                skill_by_category={skill_set_text[ii]:skill_set_value[ii]} 
                skill_by_category_list.append(skill_by_category)
    
            for jj in skill_set_title:
                skill_by_category_all={jj:skill_by_category_list}
                skill_by_category_all_list.append(skill_by_category_all)
        #print(job_list[linknumber], "\n")
        #by_page_list.append({job_list[linknumber]:skill_by_category_all_list})
        by_page_list={job_list[linknumber]:skill_by_category_all_list}
        by_page_list_all.append(by_page_list)
        #print(job_list[linknumber])
        
    all_page_list.append(by_page_list_all) ############# Her bir 30'luk paketin listeye yazdırılması
    time.sleep(3)

df=pd.DataFrame(all_page_list)   ######## Yukarıda belirlenen sayı kadar 30'luk paketlerin dosyaya yazdırılması
df.to_csv("job list{}".format(page_number))